In [0]:
import datetime as dt
print('Oi!!!')

Oi!!!


In [0]:
# Criar o diretório no Databricks
dbutils.fs.mkdirs("/FileStore/tables/")
print("Diretório criado com sucesso!")


# **1**

In [0]:
# 1
import requests

# Links diretos dos arquivos CSV no GitHub
file_urls = {
    "orders": "https://raw.githubusercontent.com/gaedesig/MVP_2/refs/heads/main/olist_orders_dataset.csv",
    "customers": "https://raw.githubusercontent.com/gaedesig/MVP_2/refs/heads/main/olist_customers_dataset.csv",
    "products": "https://raw.githubusercontent.com/gaedesig/MVP_2/refs/heads/main/olist_products_dataset.csv",
    "ord_prds": "https://raw.githubusercontent.com/gaedesig/MVP_2/refs/heads/main/olist_order_items_dataset.csv",
    "categories": "https://raw.githubusercontent.com/gaedesig/MVP_2/refs/heads/main/product_category_name_translation.csv",
    "sellers": "https://raw.githubusercontent.com/gaedesig/MVP_2/refs/heads/main/olist_sellers_dataset.csv",
    "payments": "https://raw.githubusercontent.com/gaedesig/MVP_2/refs/heads/main/olist_order_payments_dataset.csv",
    "ord_reviews": "https://raw.githubusercontent.com/gaedesig/MVP_2/refs/heads/main/olist_order_reviews_dataset.csv"
}

# Caminhos locais no Databricks para salvar os arquivos
local_paths = {name: f"/FileStore/tables/{name}.csv" for name in file_urls.keys()}

# Baixar cada arquivo e salvar no Databricks usando dbutils.fs.put
for name, url in file_urls.items():
    response = requests.get(url)
    if response.status_code == 200:  # Checar se o download foi bem-sucedido
        file_content = response.content.decode("utf-8")  # Decodificar o conteúdo do arquivo
        dbutils.fs.put(local_paths[name], file_content, overwrite=True)
        print(f"Arquivo {name} salvo em: {local_paths[name]}")
    else:
        print(f"Erro ao baixar o arquivo {name}: {response.status_code}")


Wrote 17654914 bytes.
Arquivo orders salvo em: /FileStore/tables/orders.csv
Wrote 9033957 bytes.
Arquivo customers salvo em: /FileStore/tables/customers.csv
Wrote 2379446 bytes.
Arquivo products salvo em: /FileStore/tables/products.csv
Wrote 15438671 bytes.
Arquivo ord_prds salvo em: /FileStore/tables/ord_prds.csv
Wrote 2613 bytes.
Arquivo categories salvo em: /FileStore/tables/categories.csv
Wrote 174703 bytes.
Arquivo sellers salvo em: /FileStore/tables/sellers.csv
Wrote 5777138 bytes.
Arquivo payments salvo em: /FileStore/tables/payments.csv
Wrote 14451670 bytes.
Arquivo ord_reviews salvo em: /FileStore/tables/ord_reviews.csv


In [0]:
# 2
# Listar os arquivos no diretório
dbutils.fs.ls("/FileStore/tables/")


Out[2]: [FileInfo(path='dbfs:/FileStore/tables/categories.csv', name='categories.csv', size=2613, modificationTime=1744222879000),
 FileInfo(path='dbfs:/FileStore/tables/customers.csv', name='customers.csv', size=9033957, modificationTime=1744222873000),
 FileInfo(path='dbfs:/FileStore/tables/ord_prds.csv', name='ord_prds.csv', size=15438671, modificationTime=1744222878000),
 FileInfo(path='dbfs:/FileStore/tables/ord_reviews.csv', name='ord_reviews.csv', size=14451670, modificationTime=1744222884000),
 FileInfo(path='dbfs:/FileStore/tables/orders.csv', name='orders.csv', size=17654914, modificationTime=1744222871000),
 FileInfo(path='dbfs:/FileStore/tables/payments.csv', name='payments.csv', size=5777138, modificationTime=1744222881000),
 FileInfo(path='dbfs:/FileStore/tables/products.csv', name='products.csv', size=2379446, modificationTime=1744222875000),
 FileInfo(path='dbfs:/FileStore/tables/sellers.csv', name='sellers.csv', size=174703, modificationTime=1744222879000)]

Para limpar arquivos desnecessários acima... célula seguinte

## MANUTENÇÃO

In [0]:
# Limpar todos os arquivos da pasta "/FileStore/tables/"
files = dbutils.fs.ls("/FileStore/tables/")
for file in files:
    dbutils.fs.rm(file.path, recurse=True)

print("Pasta '/FileStore/tables/' limpa com sucesso!")


Pasta '/FileStore/tables/' limpa com sucesso!


# **3** 

In [0]:
# 3
import pandas as pd

# **4** Análise inicial dos dados

In [0]:
# 4
# Ler os arquivos CSV salvos

customers_df = spark.read.format("csv").option("header", "true").load("/FileStore/tables/customers.csv")
products_df = spark.read.format("csv").option("header", "true").load("/FileStore/tables/products.csv")
order_items_df = spark.read.format("csv").option("header", "true").load("/FileStore/tables/ord_prds.csv")
category_translation_df = spark.read.format("csv").option("header", "true").load("/FileStore/tables/categories.csv")
sellers_df = spark.read.format("csv").option("header", "true").load("/FileStore/tables/sellers.csv")
payments_df = spark.read.format("csv").option("header", "true").load("/FileStore/tables/payments.csv")
order_reviews_df = spark.read.format("csv").option("header", "true").load("/FileStore/tables/ord_reviews.csv")
orders_df = spark.read.format("csv").option("header", "true").load("/FileStore/tables/orders.csv")

# Criando dataframes em pandas para melhor visualização


In [0]:
# Converter o DataFrame Spark para Pandas
orders_pandas_df = orders_df.toPandas()
customer_pandas_df = customers_df.toPandas()
order_items_pandas_df = order_items_df.toPandas()
order_reviews_pandas_df = order_reviews_df.toPandas()
payments_pandas_df = payments_df.toPandas()
products_pandas_df = products_df.toPandas()
sellers_pandas_df = sellers_df.toPandas()
category_translation_pandas_df = category_translation_df.toPandas()


# Análise inicial nos dataframes que originarão as tabelas

## Analisar nulos, valores ambíguos,  utilizando info(), describe() e outros
(ex hipotético: Ilhas das Flores <> I.Flores), possíveis outliers, etc
##

In [0]:
orders_pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [0]:
orders_pandas_df.isnull().sum()

Out[8]: order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

## order_pandas_df
Contém nulos em 3 colunas. Entender se são importantes

Entendo que esses valores nulos ñ devem influenciar.

A q, provavelmente mais influenciaria, 'order_approved_at' tem
poucos valores nulos - 160 em mais de 99 mil registros

In [0]:
customer_pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  object
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: object(5)
memory usage: 3.8+ MB


## customer_pandas_df
Não contém nenhum nulo

In [0]:
payments_pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   order_id              103886 non-null  object
 1   payment_sequential    103886 non-null  object
 2   payment_type          103886 non-null  object
 3   payment_installments  103886 non-null  object
 4   payment_value         103886 non-null  object
dtypes: object(5)
memory usage: 4.0+ MB


In [0]:
payments_pandas_df.describe()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
count,103886,103886,103886,103886,103886
unique,99440,29,5,24,29077
top,fa65dad1b0e818e3ccc5cb0e39231352,1,credit_card,1,50.00
freq,29,99360,76795,52546,324


In [0]:
payments_pandas_df['payment_value'].min()


Out[18]: '0.00'

In [0]:
payments_pandas_df['payment_value'].max()


Out[19]: '999.68'

## payment_pandas_df 
Não contém nenhum nulo

Achei valores max(999,68) e min(0,00) questionáveis. Vou avaliar na camada Gold em SQL

In [0]:
order_items_pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   order_id             112650 non-null  object
 1   order_item_id        112650 non-null  object
 2   product_id           112650 non-null  object
 3   seller_id            112650 non-null  object
 4   shipping_limit_date  112650 non-null  object
 5   price                112650 non-null  object
 6   freight_value        112650 non-null  object
dtypes: object(7)
memory usage: 6.0+ MB


In [0]:
order_items_pandas_df['price'].max()

Out[12]: '999.99'

In [0]:
order_items_pandas_df['price'].min()

Out[13]: '0.85'

In [0]:
order_items_pandas_df.describe()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
count,112650,112650,112650,112650,112650,112650,112650
unique,98666,21,32951,3095,93318,5968,6999
top,8272b63d03f5f79c56e9e4120aec44ef,1,aca2eb7d00ea1a7b8ebd4e68314663af,6560211a19b47992c3666cc44a7e94c0,2017-07-21 18:25:23,59.90,15.10
freq,21,98666,527,2033,21,2481,3707


## order_items_pandas_df
Não contém nenhum nulo

Útil na camada gold para análise

In [0]:
order_reviews_pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104162 entries, 0 to 104161
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   review_id                104161 non-null  object
 1   order_id                 101926 non-null  object
 2   review_score             101782 non-null  object
 3   review_comment_title     12005 non-null   object
 4   review_comment_message   41083 non-null   object
 5   review_creation_date     95398 non-null   object
 6   review_answer_timestamp  95377 non-null   object
dtypes: object(7)
memory usage: 5.6+ MB


In [0]:
order_reviews_pandas_df.isnull().sum()

Out[20]: review_id                      1
order_id                    2236
review_score                2380
review_comment_title       92157
review_comment_message     63079
review_creation_date        8764
review_answer_timestamp     8785
dtype: int64

In [0]:
order_reviews_pandas_df.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,None,None,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,None,None,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,None,None,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,None,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,None,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


## order_reviews_pandas_df
Contém alguns nulos. Avaliar melhor a importância disso
Algumas colunas contêm muitos valores nulos.
Entendo porém, que as principais como, review_score contêm poucos

In [0]:
category_translation_pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   product_category_name          71 non-null     object
 1   product_category_name_english  71 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


In [0]:
# Unique
pd.unique(category_translation_pandas_df['product_category_name'])

## category_translation_pandas_df
Não contém nenhum nulo e nem valores ambíguos

In [0]:
products_pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   product_id                  32951 non-null  object
 1   product_category_name       32341 non-null  object
 2   product_name_lenght         32341 non-null  object
 3   product_description_lenght  32341 non-null  object
 4   product_photos_qty          32341 non-null  object
 5   product_weight_g            32949 non-null  object
 6   product_length_cm           32949 non-null  object
 7   product_height_cm           32949 non-null  object
 8   product_width_cm            32949 non-null  object
dtypes: object(9)
memory usage: 2.3+ MB


## products_pandas_df
Vários nulos. Entender melhor o que afeta
Um ponto negativo é que esse dataframe ñ contém a descrição do produto, ficando dependendo muito da category_name, que contém centenas de 
valores nulos

In [0]:
sellers_pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   object
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: object(4)
memory usage: 96.8+ KB


In [0]:
pd.unique(sellers_pandas_df['seller_city'])

Out[30]: array(['campinas', 'mogi guacu', 'rio de janeiro', 'sao paulo',
       'braganca paulista', 'brejao', 'penapolis', 'curitiba', 'anapolis',
       'itirapina', 'tubarao', 'lauro de freitas', 'imbituba', 'brasilia',
       'porto seguro', 'guaruja', 'tabatinga', 'salto', 'tres de maio',
       'belo horizonte', 'arapongas', 'sao bernardo do campo', 'tatui',
       'garopaba', 'camanducaia', 'tupa', 'guarulhos',
       'sao jose dos pinhais', 'sao ludgero', 'sao jose', 'piracicaba',
       'porto alegre', 'congonhal', 'santo andre', 'osasco', 'valinhos',
       'joinville', 'saquarema', 'barra velha', 'petropolis',
       'santo antonio da patrulha', 'ponta grossa', 'santana de parnaiba',
       'sao carlos', 'ibitinga', 'barueri', 'caxias do sul', 'araguari',
       'contagem', 'jaragua do sul', 'lages - sc', 'bento goncalves',
       'catanduva', 'ribeirao pires', 'jaboticabal', 'echapora', 'cotia',
       'rio do sul', 'sorocaba', 'pradopolis', 'itaborai', 'mirassol',
       '

## sellers_pandas_df
Não tem valores nulos

Como em categorias, valores devem ser escolhidos de uma lista no site

%md
# Fim da análise

In [0]:
# 6
bronze_path = "/mnt/bronze/"  # Diretório para persistir os dados na camada Bronze
dbutils.fs.mkdirs(bronze_path)  # Criar o diretório no Databricks


Out[21]: True

In [0]:
# Listar os arquivos no diretório Bronze
dbutils.fs.ls("/mnt/bronze/")


Out[9]: [FileInfo(path='dbfs:/mnt/bronze/category_translation/', name='category_translation/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/bronze/customers/', name='customers/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/bronze/ord_reviews/', name='ord_reviews/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/bronze/order_items/', name='order_items/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/bronze/orders/', name='orders/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/bronze/payments/', name='payments/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/bronze/products/', name='products/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/bronze/sellers/', name='sellers/', size=0, modificationTime=0)]

In [0]:
# 7 
# Persistir o DataFrame orders_df
orders_df.write.format("delta").mode("overwrite").save("/mnt/bronze/orders")

# Persistir o DataFrame customers_df
customers_df.write.format("delta").mode("overwrite").save("/mnt/bronze/customers")

# Persistir o DataFrame products_df
products_df.write.format("delta").mode("overwrite").save("/mnt/bronze/products")


# Persistir o DataFrame order_items_df
order_items_df.write.format("delta").mode("overwrite").save("/mnt/bronze/order_items")


# Persistir o DataFrame category_translation_df
category_translation_df.write.format("delta").mode("overwrite").save("/mnt/bronze/category_translation")


# Persistir o DataFrame sellers_df
sellers_df.write.format("delta").mode("overwrite").save("/mnt/bronze/sellers")


# Persistir o DataFrame payments_df
payments_df.write.format("delta").mode("overwrite").save("/mnt/bronze/payments")


# Persistir o DataFrame order_reviews_df
order_reviews_df.write.format("delta").mode("overwrite").save("/mnt/bronze/ord_reviews")


print("DataFrames persistidos com sucesso na camada Bronze!")


DataFrames persistidos com sucesso na camada Bronze!


## Para testar resultado

In [0]:
# Exibir nomes das colunas
print(category_translation_df.columns)


['product_category_name', 'product_category_name_english']


## MANUTENÇÃO

In [0]:
# Limpar todos os arquivos da pasta "/mnt/bronze/"
files = dbutils.fs.ls("/mnt/bronze/")
for file in files:
    dbutils.fs.rm(file.path, recurse=True)

print("Pasta '/mnt/bronze/' limpa com sucesso!")

Pasta '/mnt/bronze/' limpa com sucesso!


## Ínicio tratamento dos dados para enviá-los a camada prata

# **8**

In [0]:
# 8 
# Carregar dados da camada Bronze criando dataframes
bronze_orders_df = spark.read.format("delta").load("/mnt/bronze/orders")
bronze_customers_df = spark.read.format("delta").load("/mnt/bronze/customers")
bronze_products_df = spark.read.format("delta").load("/mnt/bronze/products")
bronze_order_items_df = spark.read.format("delta").load("/mnt/bronze/order_items")
bronze_category_translation_df = spark.read.format("delta").load("/mnt/bronze/category_translation")
bronze_sellers_df = spark.read.format("delta").load("/mnt/bronze/sellers")
bronze_payments_df = spark.read.format("delta").load("/mnt/bronze/payments")
bronze_order_reviews = spark.read.format("delta").load("/mnt/bronze/ord_reviews")



In [0]:
bronze_orders_df.show(5)

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

# **9**

In [0]:
# Registrar os DataFrames como Temporary Views
bronze_orders_df.createOrReplaceTempView("bronze_orders")
bronze_customers_df.createOrReplaceTempView("bronze_customers")
bronze_products_df.createOrReplaceTempView("bronze_products")
bronze_order_items_df.createOrReplaceTempView("bronze_order_items")
bronze_category_translation_df.createOrReplaceTempView("bronze_category_translation")
bronze_sellers_df.createOrReplaceTempView("bronze_sellers")
bronze_payments_df.createOrReplaceTempView("bronze_payments")
bronze_order_reviews.createOrReplaceTempView("bronze_order_reviews")
print("Views temporárias criadas com sucesso!")


Views temporárias criadas com sucesso!


In [0]:
%sql
SELECT * FROM bronze_orders

Possível relação entre order_purchase_timestamp e order_approved_at - tempo de tela após fechar pedido

# **10**

## Se necessário...

In [0]:
%sql
DROP TABLE delta_bronze_orders
DROP TABLE delta_bronze_customers;
DROP TABLE delta_bronze_products;
DROP TABLE delta_bronze_order_items;
DROP TABLE delta_bronze_category_translation;
DROP TABLE delta_bronze_sellers;
DROP TABLE delta_bronze_payments;
DROP TABLE delta_bronze_order_reviews

In [0]:
%sql
SHOW TABLES

database,tableName,isTemporary
,bronze_category_translation,true
,bronze_customers,true
,bronze_order_items,true
,bronze_order_reviews,true
,bronze_orders,true
,bronze_payments,true
,bronze_products,true
,bronze_sellers,true


In [0]:
%sql
SELECT * FROM delta_bronze_orders

In [0]:
# Se confirmar que o diretório pode ser apagado, use:
dbutils.fs.rm("dbfs:/user/hive/warehouse/delta_bronze_orders", True) # True para deletar recursivamente
dbutils.fs.rm("dbfs:/user/hive/warehouse/delta_bronze_customers", True) 
dbutils.fs.rm("dbfs:/user/hive/warehouse/delta_bronze_products", True) 
dbutils.fs.rm("dbfs:/user/hive/warehouse/delta_bronze_order_items", True) 
dbutils.fs.rm("dbfs:/user/hive/warehouse/delta_bronze_category_translation", True) 
dbutils.fs.rm("dbfs:/user/hive/warehouse/delta_bronze_sellers", True) 
dbutils.fs.rm("dbfs:/user/hive/warehouse/delta_bronze_payments", True)   
dbutils.fs.rm("dbfs:/user/hive/warehouse/delta_bronze_order_reviews", True)  


Out[21]: True

In [0]:
%sql
-- Criar tabela Delta para bronze_orders
-- CREATE TABLE delta_bronze_orders
-- USING DELTA
-- AS SELECT * FROM bronze_orders;

-- Criar tabela Delta para bronze_customers
CREATE TABLE delta_bronze_customers
USING DELTA
AS SELECT * FROM bronze_customers;

-- Criar tabela Delta para bronze_products
CREATE TABLE delta_bronze_products
USING DELTA
AS SELECT * FROM bronze_products;

-- Criar tabela Delta para bronze_order_items
CREATE TABLE delta_bronze_order_items
USING DELTA
AS SELECT * FROM bronze_order_items;

-- Criar tabela Delta para bronze_category_translation
CREATE TABLE delta_bronze_category_translation
USING DELTA
AS SELECT * FROM bronze_category_translation;

-- Criar tabela Delta para bronze_sellers
CREATE TABLE delta_bronze_sellers
USING DELTA
AS SELECT * FROM bronze_sellers;

-- Criar tabela Delta para bronze_payments
CREATE TABLE delta_bronze_payments
USING DELTA
AS SELECT * FROM bronze_payments;

-- Criar tabela Delta para bronze_order_reviews
CREATE TABLE delta_bronze_order_reviews
USING DELTA
AS SELECT * FROM bronze_order_reviews;

-- Exibir confirmação
SHOW TABLES;


In [0]:
# Listar arquivos no diretório problemático
dbutils.fs.ls("dbfs:/user/hive/warehouse/delta_bronze_customers")

## Análise inicial dos dados na camada bronze

Entender se é melhor deixar essas consultas conclusivas na camada Gold

## Passando para a camada prata

In [0]:
# Diretório da camada Prata
silver_path = "/mnt/silver/"
dbutils.fs.mkdirs(silver_path)

Out[27]: True

In [0]:
# Listar os arquivos no diretório Silver
dbutils.fs.ls("/mnt/silver/")


Out[25]: [FileInfo(path='dbfs:/mnt/silver/category_translation/', name='category_translation/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/silver/customers/', name='customers/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/silver/order_items/', name='order_items/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/silver/order_reviews/', name='order_reviews/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/silver/orders/', name='orders/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/silver/payments/', name='payments/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/silver/products/', name='products/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/silver/sellers/', name='sellers/', size=0, modificationTime=0)]

In [0]:
# Copiar DataFrames diretamente para a camada Prata - verificar se usa essa célula
bronze_orders_df.write.format("delta").mode("overwrite").save(f"{silver_path}orders")
bronze_customers_df.write.format("delta").mode("overwrite").save(f"{silver_path}customers")
bronze_products_df.write.format("delta").mode("overwrite").save(f"{silver_path}products")
bronze_order_items_df.write.format("delta").mode("overwrite").save(f"{silver_path}order_items")
bronze_category_translation_df.write.format("delta").mode("overwrite").save(f"{silver_path}category_translation")
bronze_sellers_df.write.format("delta").mode("overwrite").save(f"{silver_path}sellers")
bronze_payments_df.write.format("delta").mode("overwrite").save(f"{silver_path}payments")
bronze_order_reviews.write.format("delta").mode("overwrite").save(f"{silver_path}order_reviews")

In [0]:
# 8 
# Carregar dados da camada Bronze criando dataframes
silver_orders_df = spark.read.format("delta").load("/mnt/silver/orders")
silver_customers_df = spark.read.format("delta").load("/mnt/silver/customers")
silver_products_df = spark.read.format("delta").load("/mnt/silver/products")
silver_order_items_df = spark.read.format("delta").load("/mnt/silver/order_items")
silver_category_translation_df = spark.read.format("delta").load("/mnt/silver/category_translation")
silver_sellers_df = spark.read.format("delta").load("/mnt/silver/sellers")
silver_payments_df = spark.read.format("delta").load("/mnt/silver/payments")
silver_order_reviews = spark.read.format("delta").load("/mnt/silver/order_reviews")



In [0]:
silver_orders_df.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [0]:
# Registrar os DataFrames como Temporary Views
silver_orders_df.createOrReplaceTempView("silver_orders")
silver_customers_df.createOrReplaceTempView("silver_customers")
silver_products_df.createOrReplaceTempView("silver_products")
silver_order_items_df.createOrReplaceTempView("silver_order_items")
silver_category_translation_df.createOrReplaceTempView("silver_category_translation")
silver_sellers_df.createOrReplaceTempView("silver_sellers")
silver_payments_df.createOrReplaceTempView("silver_payments")
silver_order_reviews.createOrReplaceTempView("silver_order_reviews")
print("Views temporárias criadas com sucesso!")


Views temporárias criadas com sucesso!


In [0]:
%sql
SHOW TABLES

database,tableName,isTemporary
default,delta_bronze_category_translation,false
default,delta_bronze_customers,false
default,delta_bronze_order_items,false
default,delta_bronze_order_reviews,false
default,delta_bronze_orders,false
default,delta_bronze_payments,false
default,delta_bronze_products,false
default,delta_bronze_sellers,false
,bronze_category_translation,true
,bronze_customers,true


In [0]:
# limpar diretório
# Se confirmar que o diretório pode ser apagado, use:
dbutils.fs.rm("dbfs:/user/hive/warehouse/delta_silver_orders", True) # True para deletar recursivamente
dbutils.fs.rm("dbfs:/user/hive/warehouse/delta_silver_customers", True) 
dbutils.fs.rm("dbfs:/user/hive/warehouse/delta_silver_products", True) 
dbutils.fs.rm("dbfs:/user/hive/warehouse/delta_silver_order_items", True) 
dbutils.fs.rm("dbfs:/user/hive/warehouse/delta_silver_category_translation", True) 
dbutils.fs.rm("dbfs:/user/hive/warehouse/delta_silver_sellers", True) 
dbutils.fs.rm("dbfs:/user/hive/warehouse/delta_silver_payments", True)   
dbutils.fs.rm("dbfs:/user/hive/warehouse/delta_silver_order_reviews", True)  

Out[36]: True

In [0]:
%sql
-- Criar tabela Delta para bronze_orders
CREATE TABLE delta_silver_orders
USING DELTA
AS SELECT * FROM silver_orders;

-- Criar tabela Delta para bronze_customers
CREATE TABLE delta_silver_customers
USING DELTA
AS SELECT * FROM silver_customers;

-- Criar tabela Delta para bronze_products
CREATE TABLE delta_silver_products
USING DELTA
AS SELECT * FROM silver_products;

-- Criar tabela Delta para bronze_order_items
CREATE TABLE delta_silver_order_items
USING DELTA
AS SELECT * FROM silver_order_items;

-- Criar tabela Delta para bronze_category_translation
CREATE TABLE delta_silver_category_translation
USING DELTA
AS SELECT * FROM silver_category_translation;

-- Criar tabela Delta para bronze_sellers
CREATE TABLE delta_silver_sellers
USING DELTA
AS SELECT * FROM silver_sellers;

-- Criar tabela Delta para bronze_payments
CREATE TABLE delta_silver_payments
USING DELTA
AS SELECT * FROM silver_payments;

-- Criar tabela Delta para bronze_order_reviews
CREATE TABLE delta_silver_order_reviews
USING DELTA
AS SELECT * FROM silver_order_reviews;

-- Exibir confirmação
SHOW TABLES;


database,tableName,isTemporary
default,delta_bronze_category_translation,false
default,delta_bronze_customers,false
default,delta_bronze_order_items,false
default,delta_bronze_order_reviews,false
default,delta_bronze_orders,false
default,delta_bronze_payments,false
default,delta_bronze_products,false
default,delta_bronze_sellers,false
default,delta_silver_category_translation,false
default,delta_silver_customers,false


In [0]:
%sql
SHOW TABLES

## Fim da camada prata

PAREI AQUI EM 09ABR25 22:38h

## Início camada gold

Transformações Avançadas:

Criar novos campos, métricas ou combinações de tabelas baseadas nos dados existentes para atender aos requisitos de análise.

Estruturação das tabelas ouro:

Definir tabelas finais ou views consolidadas para as análises, seguindo os padrões do seu projeto.

Revendo perguntas iniciais para entender que campos e consultas terei que fazer


Analisar o mercado on-line de forma sintética (Produtos mais vendidos, Produtos que mais faturaram, faixa etária que mais compra, etc.) e analítica (vendas por região, produtos por faixa etária e região, formas de pagamento por faixa etária, por período, etc.).

## 1. Qual o produto mais vendido no período registrado?

a. Como estão divididas as faixas etárias na compra desse produto?

b. Qual o peso de cada região na venda desse produto?
## 2. Qual a forma de pagamento mais usada nas transações?
a. Qual a faixe etária que mais utiliza esse método?

b. Qual o percentual de cada faixa etária que utilizam esse método?
## 3. Qual o tipo/categoria de produtos que mais vendeu nesse período?
a. Qual o percentual de cada região nas vendas desses tipos/categorias?

b. Que tipos/categorias cada região compra?
## 4. Qual produto teve a maior variação de preços durante esse período?
a. Quais os 5 produtos que tiveram a maior variação de preços no período?

b. Quais os 5 produtos que tiveram crescimento nas vendas nesse período?
## 5. Qual faixa etária compra mais?
a. Quais os tipos/categorias que cada faixa etária compra?

## 6. Qual mês mais vende?
a. Qual o período do mês mais vende?

## 7. Qual faixa etária mais usa o cartão de crédito para pagar as compras?

In [0]:
# 1. Qual o produto mais vendido no período registrado?
# Tabelas products, order e order_items


In [0]:
# 2. Qual a forma de pagamento mais usada nas transações?
# Tabelas order e payments

In [0]:
%sql
SELECT order_id, payment_type, payment_value, payment_installments FROM delta_silver_payments
ORDER BY order_id


Tentar chegar no valor total da compra comparando com payments - principalmente o campo payment_installments

Tabelas orders, order_itens, payments

In [0]:
# 3. Qual o tipo/categoria de produtos que mais vendeu nesse período?
# Tabelas products, orders, order_items


In [0]:
# 4. Qual produto teve a maior variação de preços durante esse período?
# Tabelas products, orders, order_items

In [0]:
# 5. Qual faixa etária compra mais?
# não nenhum tipo de dado que possa fornecer essa informação. Cadastro de customers ñ tem

In [0]:
# 6. Qual mês mais vende?
# Tabelas order, order_items

In [0]:
# 7. Qual faixa etária mais usa o cartão de crédito para pagar as compras?
# não nenhum tipo de dado que possa fornecer essa informação. Cadastro de customers ñ tem

## Anotações sobre análise dos dados
1 - Ñ há como definir nenhuma pergunta relacionada à faixa etária. As tabelas disponibilizadas ñ fornecem esse tipo de dado

## Fim da camada gold

In [0]:
# Refresh 1
# Refresh 2
# Refresh 3 - 08:12h
# Refresh 4  - 09:01h
# Refresh 5 - 09:20h
# Refresh 6 - 10:38h
# Refresh 7 - 10:54h
# Refresh 8 - 11:07h
# Refresh 9 - 11:42
# Refresh 1 - 12:11
# Refresh 11 - 12:40
# Refresh 12 - 12:52
# Refresh 1
# Refresh 1
# Refresh 1


## Possíveis consultas extras - início
Avaliar outras consultas durante a análise, aqui

Avaliar outras consultas em outros projetos.

## Algumas perguntas sugeridas pelo projeto do kaggle:
**Clustering:**
Some customers didn't write a review. But why are they happy or mad?

**Sales Prediction:**
With purchase date information you'll be able to predict future sales.

**Delivery Performance:**
You will also be able to work through delivery performance and find ways to optimize delivery times.

**Product Quality:**
Enjoy yourself discovering the products categories that are more prone to customer insatisfaction.

**Feature Engineering:**
Create features from this rich dataset or attach some external public information to it.

## Também
Quantidade e valores de venda por loja

In [0]:
%sql
SELECT customer_state, COUNT(DISTINCT customer_id) AS qtd_clientes
FROM delta_silver_customers
GROUP BY customer_state
ORDER BY qtd_clientes DESC
-- QTD de clientes por estado em ordem decrescente


In [0]:
%sql
SELECT product_category_name, COUNT(DISTINCT product_id) AS qtd_produtos
FROM delta_silver_products p
GROUP BY product_category_name
ORDER BY qtd_produtos DESC
-- QTD de produtos de cada categoria

In [0]:
%sql
SELECT order_id, order_item_id, product_id, price, freight_value FROM delta_bronze_order_items
WHERE order_id = "00143d0f86d6fbd9f9b38ab440ac16f5"


In [0]:
%sql
SELECT * FROM delta_bronze_products
WHERE product_id = "e95ee6822b66ac6058e2e4aff656071a"

In [0]:
%sql
SELECT order_item_id, order_id, product_id, price FROM delta_bronze_order_items
ORDER BY price DESC

In [0]:
%sql
SELECT p.product_category_name, p.product_id, oi.price
FROM delta_silver_order_items oi LEFT JOIN delta_silver_products p ON (p.product_id = oi.product_id)
WHERE price = (SELECT MAX(price) FROM delta_silver_order_items oi)
-- Encontra valor máximo e produtos com esses valores


In [0]:
%sql
SELECT order_status, COUNT(DISTINCT customer_id) AS qtd_clientes
FROM delta_silver_orders o 
WHERE( o.order_status IN ('processing','shipped','delivered')) AND
     (o.order_purchase_timestamp BETWEEN '2016-10-01' AND '2016-10-31')
GROUP BY o.order_status
HAVING qtd_clientes>1
-- Status das entregas dos pedidos em maio de 2016


In [0]:
%sql
WITH categoria_preco AS (
  SELECT oi.product_id,
    (CASE
      WHEN price < 50 THEN 'Categoria_E'
      WHEN price >= 50 AND price <100 THEN 'Categoria_D'
      WHEN price >= 100 AND price <500 THEN 'Categoria_C'
      WHEN price >= 500 AND price <1500 THEN 'Categoria_B'
      ELSE 'Categoria_A'
    END) AS categoria_produto
  FROM delta_silver_order_items oi  
    
)
SELECT categoria_produto, COUNT(DISTINCT product_id) AS qtd_produto
FROM categoria_preco cp
GROUP BY cp.categoria_produto
ORDER BY cp.categoria_produto
-- Definição e criação de categorias de produtos agrupadas por faixa de valores

In [0]:
%sql
WITH vendas AS (
  SELECT c.customer_state AS estado, p.product_category_name AS categoria, COUNT(oi.product_id) AS produtos
  FROM delta_silver_orders o LEFT JOIN delta_silver_order_items oi ON (oi.order_id = o.order_id)
                             LEFT JOIN delta_silver_products p ON (p.product_id = oi.product_id)
                             LEFT JOIN delta_silver_customers c ON (c.customer_id = o.customer_id)
  GROUP BY c.customer_state, p.product_category_name
  HAVING p.product_category_name IS NOT NULL                           
)
SELECT estado, categoria, produtos,
       SUM(produtos) OVER (PARTITION BY estado) AS total_vendas_estado,
       produtos*1.0/SUM(produtos) OVER (PARTITION BY estado) AS percentual_vendas
FROM vendas v 
ORDER BY estado, percentual_vendas DESC
-- Encontra o percentual de vendas de cada categoria de produtos em cada estado

In [0]:
%sql
WITH vendas_lojas AS (
  SELECT s.seller_id, COUNT(DISTINCT o.order_id) AS total_pedidos
  FROM delta_silver_orders o LEFT JOIN delta_silver_order_items oi ON (oi.order_id=o.order_id)
                             LEFT JOIN delta_silver_sellers s ON (s.seller_id=oi.seller_id)
  WHERE s.seller_id IS NOT NULL
  GROUP BY s.seller_id                           
)
SELECT seller_id, total_pedidos,
NTILE(4) OVER (ORDER BY total_pedidos) AS quartil
FROM vendas_lojas vl
-- Agrupa qtd total de vendas por loja e classifica em qual quartil está